In [1]:
import os
import requests
import pandas as pd
import unidecode
from tqdm import tqdm

In [2]:
tivi = pd.read_csv("Preprocessing_Tivi.csv")
tivi.head(10)

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,tivi,64.690.000 ₫,6% Off,NaN,NaN,Hồ Chí Minh,68.790.000 ₫,NaN,100%,NaN,100%,0,0,0,0,0
1,tivi,5.389.000 ₫,46% Off,130 Đã bán,(6),Hà Nội,9.890.000 ₫,NaN,100%,NaN,100%,0,0,0,0,6
2,tivi,920.486 ₫,3% Off,77 Đã bán,(26),Hà Nội,950.000 ₫,115.100 ₫,91%,95%,92%,0,0,0,2,24
3,tivi,3.989.000 ₫,50% Off,NaN,NaN,Hà Nội,7.990.000 ₫,NaN,NaN,NaN,100%,0,0,0,0,0
4,tivi,4.689.000 ₫,41% Off,16 Đã bán,NaN,Hà Nội,7.900.000 ₫,NaN,100%,NaN,100%,0,0,0,0,1
5,tivi,2.490.000 ₫,NaN,NaN,NaN,Thái Bình,NaN,141.300 ₫,NaN,NaN,100%,0,0,0,0,0
6,tivi,3.565.542 ₫,4% Off,31 Đã bán,(15),Hà Nội,3.700.000 ₫,206.800 ₫,91%,95%,92%,1,0,1,0,14
7,tivi,4.689.000 ₫,48% Off,9 Đã bán,NaN,Hà Nội,9.090.000 ₫,NaN,100%,NaN,100%,0,0,0,0,0
8,tivi,3.003.074 ₫,2% Off,21 Đã bán,(17),Hà Nội,3.070.000 ₫,198.200 ₫,91%,95%,92%,1,0,1,1,14
9,tivi,950.000 ₫,NaN,NaN,NaN,Hà Nội,NaN,198.200 ₫,86%,97%,66%,0,0,0,0,0


In [3]:
# Khởi tạo phiên làm việc với requests
s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
# Replace with long-lived credentials for production
token = "esecret_j626vlqqi569ljvpdayggnhaqj"
url = f"{api_base}/chat/completions"

In [4]:
def call_api(formatted_data):
    body = {
        "model": "meta-llama/Meta-Llama-3-70B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": "Bạn là 1 trợ lí ảo về AI và có nhiệm vụ chính là chuẩn hóa data một cách chính xác nhất"
            },
            {
                "role": "assistant",
                "content": "Nice to meet you! As a virtual assistant specializing in AI, my primary task is to standardize data with utmost accuracy. I'm here to help ensure that data is consistent, reliable, and ready for analysis or modeling.\n\nTo achieve this, I'll employ various techniques, including:\n\n1. **Data Profiling**: I'll analyze the data to understand its distribution, patterns, and quality. This helps identify errors, inconsistencies, and areas that require attention.\n2. **Data Cleansing**: I'll detect and correct errors, handle missing values, and remove duplicates or irrelevant data. This ensures that the data is accurate, complete, and reliable.\n3. **Data Transformation**: I'll convert data types, perform aggregations, and apply business rules to standardize the data format. This makes it easier to analyze and model the data.\n4. **Data Validation**: I'll check the data against a set of rules, constraints, and formats to ensure it meets the required standards.\n5. **Data Normalization**: I'll organize the data into a consistent structure, making it easier to manage, analyze, and scale.\n\nSome of the specific tasks I can perform to standardize data include:\n\n* Handling date and time formats\n* Standardizing categorical variables (e.g., converting"
            },
            {
                "role": "user",
                "content": "Mỗi mục dữ liệu được ngăn cách bởi dấu |. Không bỏ bất kì 1 mục nào, không thêm bất kỳ mục nào, hãy chuẩn hóa thông tin về dạng:\nkhông dấu, số, số, số, số, không dấu, số, số, số, số, số, số, số, số, số, số\nNếu là None thì kết quả ghi là None.\nNgăn cách các giá trị bằng dấu phẩy.Không hiển thị gì thêm, output trả về đúng 16 cột\n\ntivi  |  41.990.000 ₫  |  7% Off  |  11 Đã bán  |  NaN  |  Hồ Chí Minh  |  44.990.000 ₫ \n |  20.100 ₫  |  99%  |  100%  |  75%  |  0  |  0  |  0  |  0  |  0"
            },
            {
                "role": "assistant",
                "content": "Here is the standardized data:\n\ntivi,41990000,7,11,None,Ho Chi Minh,44990000,20100,99,100,75,0,0,0,0,0"
            },
            {
                "role": "user",
                "content": f"{formatted_data}"
            },
        ],
        "temperature": 0,
        "max_tokens": 256,
        "top_p": 1,
        "frequency_penalty": 0
    }
    response = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body)
    return response.json()

In [5]:
#Chuyển row thành chuỗi ngăn cách nhau bởi dấu |
def format_row(row):
    return '|'.join(map(str, row.values))\

#LLM code
results = []
for index, row in tqdm(tivi.iterrows(), total=tivi.shape[0], desc="Processing rows"):
    formatted_data = format_row(row)
    api_response = call_api(formatted_data)
    
    if 'choices' in api_response and api_response['choices']:
        cleaned_data = api_response['choices'][0]['message']['content'].strip()
        cleaned_data = cleaned_data.replace('Here is the standardized data:\n\n', '').replace('\n', ',')
        
        print(cleaned_data)
        results.append(cleaned_data.split(','))
    else:
        print(f"No choices found for row {index}")     #Nếu bị lỗi choices thì kết thúc vòng lặp
        break

Processing rows:   0%|          | 1/1200 [00:05<1:40:52,  5.05s/it]

tivi,64690000,6,None,None,Ho Chi Minh,68790000,None,100,None,100,0,0,0,0,0


Processing rows:   0%|          | 2/1200 [00:08<1:19:18,  3.97s/it]

tivi,5389000,46,130,6,Ha Noi,9890000,None,100,None,100,0,0,0,0,6


Processing rows:   0%|          | 3/1200 [00:11<1:16:33,  3.84s/it]

...

tivi,4349000,None,None,None,Ho Chi Minh,5950000,None,100,None,None,0,0,0,0,0


Processing rows:  95%|█████████▍| 1139/1200 [52:55<02:58,  2.92s/it]

tivi,6100000,None,None,None,Ho Chi Minh,9900000,21600,None,None,100,0,0,0,0,0


Processing rows:  95%|█████████▍| 1139/1200 [52:56<02:50,  2.79s/it]

No choices found for row 1139


In [6]:
cleaned_df = pd.DataFrame(results, columns=tivi.columns)
cleaned_df

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,tivi,64690000,6,None,None,Ho Chi Minh,68790000,None,100,None,100,0,0,0,0,0
1,tivi,5389000,46,130,6,Ha Noi,9890000,None,100,None,100,0,0,0,0,6
2,tivi,920486,3,77,26,Ha Noi,950000,115100,91,95,92,0,0,0,2,24
3,tivi,3989000,50,None,None,Ha Noi,7990000,None,None,None,100,0,0,0,0,0
4,tivi,4689000,41,16,None,Ha Noi,7900000,None,100,None,100,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,tivi,310000,None,12,4,Ha Noi,None,32700,97,96,70,0,0,0,0,4
1135,tivi,7080000,None,None,None,Ho Chi Minh,10450000,None,100,None,None,0,0,0,0,0
1136,tivi,1819452,None,28,9,Ha Noi,1860000,198200,91,95,93,0,0,0,1,9
1137,tivi,4349000,None,None,None,Ho Chi Minh,5950000,None,100,None,None,0,0,0,0,0


In [14]:
#Còn thiếu 61 rows
tivi_lack = tivi.iloc[1139:, :]

In [15]:
#Tạo API mới
s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
# Replace with long-lived credentials for production
token = "esecret_sbtbku3acce7j3huyaz24zbixp"
url = f"{api_base}/chat/completions"

In [16]:
def call_api_lack(formatted_data):
    body = {
    "model": "meta-llama/Meta-Llama-3-70B-Instruct",
    "messages": [
        {
        "role": "user",
        "content": "Bạn là trợ lí ảo về AI và có khả năng chuẩn hóa data 1 cách chính xác nhất"
        },
        {
        "role": "assistant",
        "content": "Nice to meet you! As a virtual assistant specializing in AI and data standardization, I'm here to help you with any data-related tasks. My capabilities include:\n\n1. **Data Cleaning**: I can identify and correct errors, inconsistencies, and inaccuracies in your data, ensuring it's reliable and trustworthy.\n2. **Data Normalization**: I can transform your data into a consistent format, making it easier to analyze, process, and store.\n3. **Data Standardization**: I can apply standardized formats, codes, and conventions to your data, enabling seamless integration and comparison across different systems and datasets.\n4. **Data Transformation**: I can convert your data from one format to another, such as from CSV to JSON or from Excel to SQL, while preserving its integrity and structure.\n5. **Data Quality Control**: I can perform checks and balances to ensure your data meets specific quality standards, detecting and correcting errors, duplicates, or inconsistencies.\n6. **Data Enrichment**: I can enhance your data by adding missing information, filling gaps, or appending relevant data from external sources, making it more comprehensive and valuable.\n7. **Data Validation**: I can verify the accuracy and consistency of your data against predefined rules, constraints, or reference data, ensuring it conforms to"
        },
        {
        "role": "user",
        "content": "\"Mỗi mục dữ liệu được ngăn cách bởi dấu |. Không bỏ bất kì 1 mục nào, không thêm bất kỳ mục nào, hãy chuẩn hóa thông tin về dạng:\nkhông dấu, số, số, số, số, không dấu, số, số, số, số, số, số, số, số, số, số\nNếu là None thì kết quả ghi là None.\nNgăn cách các giá trị bằng dấu phẩy.Không giải thích output, output trả về đúng 16 cột\n\ntivi  |  41.990.000 ₫  |  7% Off  |  11 Đã bán  |  NaN  |  Hồ Chí Minh  |  44.990.000 ₫ |  20.100 ₫  |  99%  |  100%  |  75%  |  0  |  0  |  0  |  0  |  0\""
        },
        {
        "role": "assistant",
        "content": "Here is the standardized output:\n\ntivi,41990000,7,11,None,Ho Chi Minh,44990000,20100,99,100,75,0,0,0,0,0"
        },
        {
            "role": "user",
            "content": f"{formatted_data}"
        },
    ],
    "temperature": 0,
    "max_tokens": 256,
    "top_p": 1,
    "frequency_penalty": 0
    }
    response = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body)
    return response.json()

In [17]:
results_lack = []

for index, row in tqdm(tivi_lack.iterrows(), total=tivi_lack.shape[0], desc="Processing rows"):
    formatted_data = format_row(row)
    api_response = call_api(formatted_data)
    
    if 'choices' in api_response and api_response['choices']:
        cleaned_data = api_response['choices'][0]['message']['content'].strip()
        cleaned_data = cleaned_data.replace('Here is the standardized data:\n\n', '').replace('\n', ',')
        print(cleaned_data)
        results_lack.append(cleaned_data.split(','))
    else:
        print(f"No choices found for row {index}")
        break

Processing rows:   2%|▏         | 1/61 [00:04<04:13,  4.23s/it]

tivi,6490000,None,None,None,Ho Chi Minh,11490000,131600,None,None,100,0,0,0,0,0


Processing rows:   3%|▎         | 2/61 [00:07<03:29,  3.56s/it]

tivi,2790000,None,None,None,Ho Chi Minh,None,46000,100,None,66,0,0,0,0,0


Processing rows:   5%|▍         | 3/61 [00:10<03:16,  3.38s/it]

...

tivi,6150000,None,None,None,Ho Chi Minh,9990000,None,97,98,89,0,0,0,0,0


Processing rows:  98%|█████████▊| 60/61 [02:48<00:02,  2.56s/it]

tivi,4250000,None,None,None,Ha Noi,4990000,135500,100,40,None,0,0,0,0,0


Processing rows: 100%|██████████| 61/61 [02:50<00:00,  2.80s/it]

tivi,2650000,None,None,None,Ho Chi Minh,None,45100,100,None,50,0,0,0,0,0


In [18]:
lack_df = pd.DataFrame(results_lack, columns=tivi_lack.columns)
lack_df.head()

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,tivi,6490000,None,None,None,Ho Chi Minh,11490000,131600,None,None,100,0,0,0,0,0
1,tivi,2790000,None,None,None,Ho Chi Minh,None,46000,100,None,66,0,0,0,0,0
2,tivi,5590000,None,5,None,Ho Chi Minh,None,None,99,None,100,0,0,0,0,0
3,tivi,2350000,None,18,9,Ha Noi,None,135500,91,95,93,0,1,0,1,7
4,tivi,1850000,None,38,14,Ha Noi,None,198200,91,95,93,1,0,1,4,9


In [19]:
final_data = pd.concat([cleaned_df, lack_df], ignore_index=True)
final_data 

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,tivi,64690000,6,None,None,Ho Chi Minh,68790000,None,100,None,100,0,0,0,0,0
1,tivi,5389000,46,130,6,Ha Noi,9890000,None,100,None,100,0,0,0,0,6
2,tivi,920486,3,77,26,Ha Noi,950000,115100,91,95,92,0,0,0,2,24
3,tivi,3989000,50,None,None,Ha Noi,7990000,None,None,None,100,0,0,0,0,0
4,tivi,4689000,41,16,None,Ha Noi,7900000,None,100,None,100,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,tivi,5390000,34,9,Ho Chi Minh,5590000,0,96,100,100,2,0,0,0,7,None
1196,tivi,2730000,None,None,None,Ho Chi Minh,None,38000,None,None,None,0,0,0,0,0
1197,tivi,6150000,None,None,None,Ho Chi Minh,9990000,None,97,98,89,0,0,0,0,0
1198,tivi,4250000,None,None,None,Ha Noi,4990000,135500,100,40,None,0,0,0,0,0


In [20]:
final_data.to_csv("LLM_Tivi.csv", index=False)